## 输入一句话,预测后面的一个单词(只预测一个)
## 这个写的有点迷,能运行(大家凑合看吧.....)



In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

In [2]:
sentence = (
    'GitHub Actions makes it easy to automate all your software workflows '
    'from continuous integration and delivery to issue triage and more END'
)
s1 = 'GitHub Actions makes it easy to automate all your software workflows from continuous integration and delivery to issue triage and more END'
print(sentence == s1)

print('sentence总长度是:', len(list(s1.split())))
print('sentence不同单词的个数是:', len(set(list(s1.split()))))

True
sentence总长度是: 22
sentence不同单词的个数是: 20


In [3]:
# word_list去重
word_list = set(list(s1.split()))
print('字典是:', word_list)
print('字典长度是:', len(word_list))
# 字典长度是: 20 也就是20分类问题

字典是: {'continuous', 'makes', 'Actions', 'more', 'to', 'all', 'workflows', 'and', 'automate', 'triage', 'it', 'integration', 'your', 'from', 'software', 'issue', 'easy', 'GitHub', 'delivery', 'END'}
字典长度是: 20


In [4]:
## 构建字典索引
word2index = {v: k for k, v in enumerate(word_list)}
index2word = {k: v for k, v in enumerate(word_list)}
print(word2index)
print(index2word)

{'continuous': 0, 'makes': 1, 'Actions': 2, 'more': 3, 'to': 4, 'all': 5, 'workflows': 6, 'and': 7, 'automate': 8, 'triage': 9, 'it': 10, 'integration': 11, 'your': 12, 'from': 13, 'software': 14, 'issue': 15, 'easy': 16, 'GitHub': 17, 'delivery': 18, 'END': 19}
{0: 'continuous', 1: 'makes', 2: 'Actions', 3: 'more', 4: 'to', 5: 'all', 6: 'workflows', 7: 'and', 8: 'automate', 9: 'triage', 10: 'it', 11: 'integration', 12: 'your', 13: 'from', 14: 'software', 15: 'issue', 16: 'easy', 17: 'GitHub', 18: 'delivery', 19: 'END'}


In [5]:
# sen_list句子中所有的单词，不去重
sen_list = list(s1.split())
print(len(sen_list))  # 22
max_len = 22
print(max_len)


## 构建one-hot形式的字典索引
# 句子长度是22 一共有20种不同的词
# 也就是有21个输入,对应21个输出
# 此时<END>对应的是16
def make_sentence(s1):
    input_batch = []
    target_batch = []
    words = sentence.split()

    for i in range(max_len - 1):
        input1 = [word2index[n] for n in words[:(i + 1)]]
        input1 = input1 + [0] * (max_len - len(input1))
        # 占位符 句子长度是22,输入(最大也就是固定的)有21位
        # 预测要从一号(也就是第2个)  开始预测
        target1 = word2index[words[i + 1]]
        # 20表示字典长度
        input_batch.append(np.eye(20)[input1])
        target_batch.append(target1)
    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

22
22


In [6]:
np.eye(20).shape

(20, 20)

In [7]:
input_batch, target_batch = make_sentence(s1)
print(input_batch, input_batch.shape, len(input_batch))
# input_batch的形状是([21, 21, 20])
# 21:sentence总长度是: 22个单词最多有21个输入,也就是有21个句子
# 21:每个输入都经过添0后,变为21维,每个句子有21个单词
# 20:字典长度是: 20,每个单词被编码为20维

print(target_batch, target_batch.shape, len(target_batch))
# torch.Size([21])
# 21个输入有21个输出

tensor([[[0., 0., 0.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0., 

D:\ProgramData\Anaconda3\envs\pycharmshiyong\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)


In [8]:
dataset11 = Data.TensorDataset(input_batch, target_batch)
dataloader = Data.DataLoader(dataset=dataset11, batch_size=3,
                             shuffle=False, drop_last=True)

In [9]:
for data, target in dataloader:
    print(data, target)
    print('data.shape:', data.shape, 'target.shape:', target.shape)
    print('----')
# data.shape: torch.Size([3, 21, 20]) target.shape: torch.Size([3])
#  3:DataLoader中batch_size=3,也就是有三个句子
#  21: 每个句子有21个单词
#  20: 每个单词被编码为20维


tensor([[[0., 0., 0.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]]) tensor([ 2,  1, 10])
data.shape: torch.Size([3, 22, 20]) target.shape: torch.Size([3])
----
tensor([[[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],

<img src="./lstm.jpg">

In [10]:
# 双向LSTM
n_hidden = 8


class MyBiLstm(nn.Module):
    def __init__(self):
        super(MyBiLstm, self).__init__()
        self.lstm = nn.LSTM(input_size=20,
                            hidden_size=n_hidden, bidirectional=True,
                            num_layers=1)  # num_layers=1一层
        self.quanlianjie = nn.Linear(in_features=n_hidden * 2,
                                     out_features=len(word_list))
        # word_list是字典长度,也就是去重后有多少种不同的字符

    # data.shape: torch.Size([3, 21, 20])
    def forward(self, x):
        batch_size =x.shape[0]   # x.shape[0]  3
        print("batch_size",batch_size)
        input22 = x.transpose(0, 1)
        #input22:[max_len, batch_size, n_class(每个单词被编码为20维)]

        h_0 = torch.zeros(1 * 2, batch_size, n_hidden)
        # [层数*方向数, batch_size, n_hidden] # n_hidden = 8
        # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        c_0 = torch.zeros(1 * 2, batch_size, n_hidden)
        #
        output, (hi, ci) = self.lstm(input22, (h_0, c_0))
        ## 只要最后的输出 output是所有输出的集合,包括[output1,output2...]
        # 此处只要最后一个
        #output: [seq_len, batch,hidden_size *num_directions]
        output = output[-1]
        model = self.quanlianjie(output)
        return model

<img src="./lstm1.jpg">

In [11]:
model11 = MyBiLstm()
print(model11)


MyBiLstm(
  (lstm): LSTM(20, 8, bidirectional=True)
  (quanlianjie): Linear(in_features=16, out_features=20, bias=True)
)


In [12]:
## 定义损失函数&优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model11.parameters(),lr=0.01)


In [13]:
### 训练
for epoch in range(10):
    for x, y in dataloader:
        ## data和target在上面已经分析过了
        pred = model11(x)  # 这里的x也就代表,上面的模型定义中的forward(self, x),也就是batch_size(3)个一组的自变量数据
        print(x.shape)
        loss = loss_fn(pred, y)
        print(epoch, loss)
        ###
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        optimizer.zero_grad()  # 梯度清零

batch_size 3
torch.Size([3, 22, 20])
0 tensor(2.9104, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(2.9402, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(2.9756, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(3.0353, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(3.2255, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(3.1240, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
0 tensor(3.1508, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
1 tensor(2.7990, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
1 tensor(2.8710, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
1 tensor(2.9452, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
1 tensor(3.0153, grad_fn=<NllLossBackward0>)
batch_size 3
torch.Size([3, 22, 20])
1 tensor(3.1915, grad_fn=<NllLossBackward0>)
batch_size 3
tor

In [14]:
predict123 = model11(input_batch).data.max(1, keepdim=True)[1]
print(predict123)
print([index2word[n.item()] for n in predict123.squeeze()])

batch_size 21
tensor([[ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 4],
        [ 7],
        [19]])
['to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'to', 'and', 'END']


In [ ]:
###########################################################################

In [15]:
# sentence = (
#     'GitHub Actions makes it easy to automate all your software workflows '
#     'from continuous integration and delivery to issue triage and more END'
# )

# ##test
# input = [sen.split()[:2] for sen in sentences]
# print(input)


<img src="./tset.jpg">

In [16]:
## 构建测试集  要构建input22 h_0 c_0
input_ceshi = 'makes it easy'
print(input_ceshi, type(input_ceshi), len(input_ceshi))
# h_ceshi = torch.zeros(2, 3, 8)
# c_ceshi = torch.zeros(2, 3, 8)

makes it easy <class 'str'> 13


In [17]:
## 将GitHub Actions makes it...转为指定的格式
# 这里的输出,可能会带有前面函数里的,这里加---来做区别
input_batch88, target_batch88 = make_sentence(input_ceshi)
print('---', input_batch88, input_batch88.shape)  #torch.Size([21, 21, 20])
print('--', target_batch88, target_batch88.shape)  #torch.Size([21])

--- tensor([[[0., 0., 0.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 

In [18]:
dataset88 = Data.TensorDataset(input_batch88, target_batch88)
dataloader88 = Data.DataLoader(dataset=dataset88, batch_size=3,
                               shuffle=False, drop_last=True)

In [19]:
for x, y in dataloader88:
    predict88 = model11(x)

batch_size 3
batch_size 3
batch_size 3
batch_size 3
batch_size 3
batch_size 3
batch_size 3


In [20]:
print(predict88, predict88.shape)

tensor([[-0.0331,  0.0312, -0.2143, -0.5183,  0.4288,  0.0123, -0.0616,  0.4196,
         -0.0742, -0.3536,  0.0791, -0.1539, -0.0099, -0.1395, -0.0863, -0.2741,
         -0.0565, -1.6993, -0.1560, -0.4935],
        [-0.1412, -0.0046, -0.4549, -0.0214,  0.2647, -0.0696, -0.1308,  0.5154,
         -0.0592,  0.0040,  0.1352, -0.2248, -0.0203, -0.1215, -0.1419, -0.0502,
         -0.1509, -1.1719, -0.1177,  0.2544],
        [-0.2457, -0.0204, -0.6596,  0.4588,  0.1151, -0.1528, -0.2163,  0.5383,
         -0.0196,  0.3041,  0.1751, -0.2678, -0.0287, -0.0762, -0.1436,  0.1523,
         -0.2429, -0.6796, -0.1117,  0.9174]], grad_fn=<AddmmBackward0>) torch.Size([3, 20])


In [21]:
predict = predict88.data.max(1, keepdim=True)[1]
print(predict)

tensor([[ 4],
        [ 7],
        [19]])


In [22]:
print([index2word[n.item()] for n in predict.squeeze()])

['to', 'and', 'END']
